In [8]:
import pandas as pd
df_stc = pd.read_excel(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\database\data\stc\stc.xlsx")
df_stc = df_stc.drop_duplicates()
df_stc['drs:stcHolder'] = df_stc['drs:stcHolder'].map(lambda x: x.replace(", Inc.","").replace(", Inc","").replace(" Inc.","").replace(" Inc",""))

print(df_stc.shape)

(77661, 26)


### Cleaning the STC with fields that get incorrectly decoded
TODO

### Decode STCs text

In [7]:
import re

def get_block(text, pattern_start, pattern_end):
    try:
        match_start = re.search(pattern_start, text, re.IGNORECASE)
        match_end = re.search(pattern_end, text[match_start.span(0)[1]:], re.IGNORECASE)

        return text[match_start.span(0)[1]:(match_end.span(0)[0]+match_start.span(0)[1])]
    except:
        return ''

In [6]:
import re

def get_all_descriptions(pdf_pages):
    descriptions = []

    # First page
    descriptions.append(get_block(pdf_pages[0],
                                  '(\n|\n\r|\r|\(| )Description of (the )?Type Design Change:?\.?(\n|\n\r|\r|\)| )',
                                  '(Limitations |[a-z ]{5,10}ions and Cond|\(Description |\(See continuation)'))
    # Other pages
    final_del = '- - -|\. \. \. |\* \* \* |END|---|\.\.\.|\*\*\*|Certification Basis|Certification Basis|\(See continuation|\(cont'
    for n in range(1, len(pdf_pages)):
        descriptions.append(get_block(pdf_pages[n],
                                      '(\n|\n\r|\r|\(| )Description of (the )?Type Design Change:?\.? (\(con[a-z ]+\)|con[a-z ]+):?(\n|\n\r|\r|\)| )',
                                      '(\n|\n\r|\r|\(| )(Limitations and Conditions (\(con[a-z ]+\)|con[a-z ]+):'+'|'+final_del+')'))
        

    return descriptions

In [4]:
import re

def get_all_limitations(pdf_pages):
    limitations = []

    # First page
    limitations.append(get_block(pdf_pages[0],
                                  '(\n|\n\r|\r)(Limitations and Conditions|[a-z ]{5,10}ions and Cond[a-z ]{5,10}):?\.?(\n|\n\r|\r| )',
                                  '(\(Limitations |(\n|\n\r|\r)This certificat|\(See )'))
    # Other pages     
    final_del = '---|\.\.\.|\*\*\*|Certification Basis|Certification Basis|\(See continuation|\(cont'
    for n in range(1, len(pdf_pages)):
        final_del = '- - -|\. \. \. |\* \* \* |---|\.\.\.|\*\*\*|Certification Basis|Certification Basis|\(See continuation|\(cont'
        limitations.append(get_block(pdf_pages[n],
                                      '(\n|\n\r|\r|\(| )Limitations and Conditions:?\.? (\(con[a-z ]+\)|con[a-z ]+):?(\n|\n\r|\r|\)| )', 
                                      "(\n|\n\r|\r|\(| )("+final_del+")"))
    return limitations

In [5]:
import re

def split_limitations(text):
    split_text = re.split('(\\n|\s)[1]{0,1}[0-9]\.\s', text)
    split_text = [re.sub('\n', ' ', val) for val in split_text if len(val)>5]
    return split_text

### Example to decode stc

#### From pdf

In [78]:
import pdfplumber

with pdfplumber.open(r'C:\Users\victor\Documents\DeepLearning\FAA NLP Project\database\data\stc\pdf\SA01661SE__Current__7F91E1600D295CEF8625715400581B6B.pdf') as pdf:
    pdf_pages = [str(this_page.extract_text())+'\n' for this_page in pdf.pages]

    print(get_all_descriptions(pdf_pages))
    print("=======================")
    print(get_all_limitations(pdf_pages))
    print("=======================")
    print(pdf_pages)

'''
SA00001SE__Historical__7B567CD4F182FBDB85256CC1007DC6F6

SR00592DE__Current__1331A416DAE36E99862588A7004BFFD0
SA1771CE-D__Current__84229556CA57EA138625861E007A36C9
SA2196CE__Current__36F896ABA7701168862585180058641A

ST04436AT (2 OF 2)__Current__8C9E2E584973558B86258876006E8DB7

SR03913NY__Current__C9692170F9AEC5C6862585EC00565F13

SA760CE__Historical__262337959FF88250862572C70055A8F3

ST02184LA-D__Current__8A332D420524BE9F862575FC006B4BAF

SA01661SE__Current__7F91E1600D295CEF8625715400581B6B
'''

['']
['Data pertaining to this modification are considered inadequate for\nduplication in other aircraft. This approval is therefore limited to the installation in Waco UKC airplane\nserial number 3978, registration N14611 only. A copy of this certificate, and the Airplane Flight Manual\nSupplement dated March 31, 2006 shall be maintained as part of the permanent records for the\nmodified aircraft.\n']
['UnitedStatesofAmerica\nDepartmentofTransportation_FederalAviationAdministration\nSupplementalTypeCertificate\nNumber\nSA01661SE\nThis certificate, issued to Olde Thyme Aviation\n21704 141st Ave. SW\nVashon Island, WA 98070\ncertifies that the change in the type design for the following product with the limitations and conditions\ntherefor as specified hereon meets the airworthiness requirements of Type Certificate ATC 528.\nOriginal Product\uf0beType Certificate Number: ATC 528\nMake: Waco\nModel: UKC\nDescription of the Type Design Change: Install a Continental W670-23 engine in the a

'\nSR00592DE__Current__1331A416DAE36E99862588A7004BFFD0\nSA1771CE-D__Current__84229556CA57EA138625861E007A36C9\nSA2196CE__Current__36F896ABA7701168862585180058641A\n\nST04436AT (2 OF 2)__Current__8C9E2E584973558B86258876006E8DB7\n\nSR03913NY__Current__C9692170F9AEC5C6862585EC00565F13\n\nSA760CE__Historical__262337959FF88250862572C70055A8F3\n\nST02184LA-D__Current__8A332D420524BE9F862575FC006B4BAF\n\nSA01661SE__Current__7F91E1600D295CEF8625715400581B6B\n'

#### From text

In [45]:
with open(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\database\data\stc\text-from-pdf\SA00027MC-D__Current__AB80219E04E9EC478625762000544966.txt","r") as my_text:
    pdf_pages = my_text.read().split("\n\n")

    print(get_all_descriptions(pdf_pages))
    print("=======================")
    print(get_all_limitations(pdf_pages))
    print("=======================")
    print(pdf_pages)

'''
SA00001SE__Historical__7B567CD4F182FBDB85256CC1007DC6F6

SR00592DE__Current__1331A416DAE36E99862588A7004BFFD0
SA1771CE-D__Current__84229556CA57EA138625861E007A36C9
SA2196CE__Current__36F896ABA7701168862585180058641A

ST04436AT (2 OF 2)__Current__8C9E2E584973558B86258876006E8DB7

SR03913NY__Current__C9692170F9AEC5C6862585EC00565F13

SA760CE__Historical__262337959FF88250862572C70055A8F3

ST02184LA-D__Current__8A332D420524BE9F862575FC006B4BAF

SA01661SE__Current__7F91E1600D295CEF8625715400581B6B
'''

['Installation of a L3 Communciations Cockpit Voice and Flight Data Recorder (CVDR) in accordance with FAA\napproved ARINC Master Data List, Document No. 74-2370X001 Revision “A”, dated July 16, 2009 or later FAA\napproved revisions.\n']
['2.\n1. Compatibility of this design change with previously approved modifications must be determined by installer.\nInstructions for Continued Airworthiness, Document 74-2370C001 entitled: "Viking DHC-6-300 Twin Otter\nCockpit Voice / Data Recorder Instructions for Continued Airworthiness", Revision A, dated July 8, 2009, or\nlater FAA accepted revision is required for this installation.\n3.\nIf the holder agrees to permit another person to use this certificate to alter the product, the holder shall give\nthe other person written evidence of the permission.\n4.\nA copy of this certificate and FAA approved Airplane Flight Manual Supplement, Document 74-2370A001,\nRevision IR, dated July 16, 2009 or later FAA approved revision for the Cockpit Voice / D

'\nSA00001SE__Historical__7B567CD4F182FBDB85256CC1007DC6F6\n\nSR00592DE__Current__1331A416DAE36E99862588A7004BFFD0\nSA1771CE-D__Current__84229556CA57EA138625861E007A36C9\nSA2196CE__Current__36F896ABA7701168862585180058641A\n\nST04436AT (2 OF 2)__Current__8C9E2E584973558B86258876006E8DB7\n\nSR03913NY__Current__C9692170F9AEC5C6862585EC00565F13\n\nSA760CE__Historical__262337959FF88250862572C70055A8F3\n\nST02184LA-D__Current__8A332D420524BE9F862575FC006B4BAF\n\nSA01661SE__Current__7F91E1600D295CEF8625715400581B6B\n'

### Extract texts to dataframe

In [ ]:
import os
import glob
import pandas as pd
import pdfplumber

dl_dir = r'C:\Users\victor\Documents\DeepLearning\FAA NLP Project\database\data\stc\pdf'

list_of_real_pdfs = [val for val in glob.glob(os.path.join(dl_dir,"*.pdf")) if os.path.getsize(val)>200]
pdf_content = [[]]

for n, pdf in enumerate(list_of_real_pdfs):
    split_name = os.path.basename(pdf[:-4]).split("__")

    try:
        with pdfplumber.open(pdf) as opened_pdf:        
            pdf_pages = [str(this_page.extract_text())+'\n' for this_page in opened_pdf.pages]

            pdf_content.append([split_name[0],
                                split_name[1],
                                split_name[2],
                                get_all_descriptions(pdf_pages),
                                get_all_limitations(pdf_pages),
                                df_stc[df_stc["documentGuid"]==split_name[2]]["drs:title"].values[0],
                                df_stc[df_stc["documentGuid"]==split_name[2]]["drs:stcAplicationDate"]])
            
        break

    except:
        print(pdf)

df_pdf_content = pd.DataFrame(pdf_content,
                            columns=["drs:chronicleId",
                                    "drs:status",
                                    "documentGuid",
                                    "description",
                                    "limitation",
                                    "drs:title",
                                    "drs:stcAplicationDate"])    

df_pdf_content.to_excel('pdfs_with_content.xlsx')

In [9]:
import os
import glob
import pandas as pd
import pdfplumber

dl_dir = r'C:\Users\victor\Documents\DeepLearning\FAA NLP Project\database\data\stc\text-from-pdf'

list_of_real_pdfs = [val for val in glob.glob(os.path.join(dl_dir,"*.txt")) if os.path.getsize(val)>0]
pdf_content = [[]]

for n, file in enumerate(list_of_real_pdfs):
    split_name = os.path.basename(file[:-4]).split("__")

    try:
        with open(file,"r") as my_text:
            pdf_pages = my_text.read().split("\n\n")

            pdf_content.append([split_name[0],
                                split_name[1],
                                split_name[2],
                                get_all_descriptions(pdf_pages),
                                get_all_limitations(pdf_pages),
                                df_stc[df_stc["documentGuid"]==split_name[2]]["drs:title"].values[0],
                                df_stc[df_stc["documentGuid"]==split_name[2]]["drs:stcAplicationDate"]])
            
        if n>400000:    
            break

    except:
        print(file)

df_pdf_content = pd.DataFrame(pdf_content,
                            columns=["drs:chronicleId",
                                    "drs:status",
                                    "documentGuid",
                                    "description",
                                    "limitation",
                                    "drs:title",
                                    "drs:stcAplicationDate"])    

df_pdf_content = df_pdf_content.tail(-1)

df_pdf_content.to_excel('pdfs_with_content.xlsx')